In [ ]:
# !python3 -m pip install Pillow

# Create 2 separate input embed layers
# Connect to common LSTM-LSTM
# Branch out and predict IOSIze and Offset differently

In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb


from tensorflow.python.client import device_lib
import numpy as np
import csv
import pandas as pd
import sys
import os
import glob
import tensorflow as tf


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "3"



Using TensorFlow backend.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import numpy as np
import pandas as pd
import glob
import csv
import sys
import os
import math
import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
from sklearn import metrics
from keras.layers.core import Dense, Activation
from keras.callbacks import ModelCheckpoint
from numpy import insert
from sklearn import preprocessing
from collections import Counter
from numpy import array
from numpy import hstack
from keras.preprocessing.sequence import TimeseriesGenerator




path  = r"/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/SYSTOR/part27/" 

names = ['TimeStamp','Response','IOType','LUN','ByteOffset','Size']



all_files = glob.glob(os.path.join(path, "2016030819-LUN2.csv"))
print(all_files)




['/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/SYSTOR/part27/2016030819-LUN2.csv']


In [4]:
df = pd.read_csv(all_files[0],engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False) 
names = ['TimeStamp','Response','IOType','LUN','ByteOffset','Size']   
df.columns = names
print (len(df))
df['ByteOffset_Delta'] = df['ByteOffset'] - df['ByteOffset'].shift(-1)
df = df.drop(df.index[-1])

total_classes = len(Counter(df['ByteOffset_Delta']))
x = Counter(df['ByteOffset_Delta'])
vals = {}
vals =  x.most_common(1000)
bo_list = []
coverage = 0
for key,value in vals:
    bo_list.append(key)
    coverage = coverage + value
print(len(bo_list))
print("Percentage Coverage")
print((coverage/len(df))*100) 
total_classes = len(Counter(df['ByteOffset']))
x = Counter(df['ByteOffset'])
vals = {}
vals =  x.most_common(1000)
bo_list = []
coverage = 0
for key,value in vals:
    bo_list.append(key)
    coverage = coverage + value
    
print(len(bo_list))
print("Percentage Coverage Offset")
print((coverage/len(df))*100)
    
 



2902328
1000
Percentage Coverage
68.94633168488595
1000
Percentage Coverage Offset
69.80925994899955


In [11]:
from collections import Counter
len(Counter(df['Size']))

259

In [12]:
df['ByteOffset_Delta'] = df['ByteOffset_Delta'].fillna(0)

a = df['ByteOffset_Delta'].unique().tolist()
operation_id_map = {}
for i,id in enumerate(a): operation_id_map[id] = i 
df['ByteOffset_Delta_class'] = df['ByteOffset_Delta'].map(lambda x: operation_id_map[x])


In [13]:
a = df['Size'].unique().tolist()
size_id_map = {}
for i,id in enumerate(a): size_id_map[id] = i 
df['Size_class'] = df['Size'].map(lambda x: operation_id_map[x])

In [14]:
#Sorting df by TimeStamp

df = df.sort_values(by=['TimeStamp'])
df.reset_index(inplace=True, drop=True)
print(df.dtypes)
print(len(df))

TimeStamp                 float64
Response                  float64
IOType                     object
LUN                         int64
ByteOffset                  int64
Size                        int64
ByteOffset_Delta          float64
ByteOffset_Delta_class      int64
Size_class                  int64
dtype: object
2902327


In [15]:
from collections import Counter
x = Counter(df['ByteOffset_Delta_class'])
vals = {}
vals =  x.most_common(1000)
bo_list = []

for x in vals:
    bo_list.append(x[0])
        
count = 0
label_list = []

while (count < len(df)):
    x = df['ByteOffset_Delta_class'].iloc[count]
    if x in bo_list:
        label_list.append(x)
    else:
        label_list.append(999999)
    count= count + 1
    
ByteOffset_Delta_class_backup  = df['ByteOffset_Delta_class'] 
df['ByteOffset_Delta_class']  = label_list
print(len(Counter(df['ByteOffset_Delta_class'])))
    
a = df['ByteOffset_Delta_class'].unique().tolist()
bo_map = {}
for i,id in enumerate(a): bo_map[id] = i 
df['ByteOffset_Delta_Class_1001'] = df['ByteOffset_Delta_class'].map(lambda x: bo_map[x])
    
label_list = df['ByteOffset_Delta_Class_1001'] 
    
df['ByteOffset_Delta_Class_1001']  = label_list
    

1001


In [16]:
# amin, amax = df['Size'].min(), df['Size'].max()
# a = df['Size'].tolist()
# for i, val in enumerate(a):
#     a[i] = (val-amin) / (amax-amin)
    
# df['Size'] = a

In [17]:
df.head(5)

,TimeStamp,Response,IOType,LUN,ByteOffset,Size,ByteOffset_Delta,ByteOffset_Delta_class,Size_class,ByteOffset_Delta_Class_1001
0,1.457431e+09,0.005598,R,2,5119596273664,4096,1.342012e+12,999999,919,0
1,1.457431e+09,0.004451,R,2,5119596236800,4096,3.857761e+12,999999,919,0
2,1.457431e+09,0.000162,W,2,3777583883264,12288,-3.002076e+08,1,175,1
3,1.457431e+09,0.000139,W,2,3777884090880,8192,-1.444479e+09,2,570,2
4,1.457431e+09,0.000148,W,2,3779328570368,8192,1.596548e+09,3,570,3


In [18]:
df.dtypes

TimeStamp                      float64
Response                       float64
IOType                          object
LUN                              int64
ByteOffset                       int64
Size                             int64
ByteOffset_Delta               float64
ByteOffset_Delta_class           int64
Size_class                       int64
ByteOffset_Delta_Class_1001      int64
dtype: object

In [19]:
# Drop unnecessary coloumns

print("Before drop: {}".format(df.columns))
df.drop(df.columns[[0,1,2,3,4,5,6,7]], axis=1,inplace=True) 


Before drop: Index(['TimeStamp', 'Response', 'IOType', 'LUN', 'ByteOffset', 'Size',
       'ByteOffset_Delta', 'ByteOffset_Delta_class', 'Size_class',
       'ByteOffset_Delta_Class_1001'],
      dtype='object')


In [20]:
df.dtypes

Size_class                     int64
ByteOffset_Delta_Class_1001    int64
dtype: object

In [21]:
df_backup = df

In [22]:
# Split to train, validate and test

# Finding the value 75th percentile of TimeStamp

import math


training_pt_1 = math.floor((len(df)*0.75)) 


data_train = df[:training_pt_1]
data_test = df[training_pt_1+1:]

print(len(data_train))
print(len(data_test))

2176745
725581


In [23]:
data_train =df[:training_pt_1]['ByteOffset_Delta_Class_1001'].values.tolist()
data_test = df[training_pt_1+1:]['ByteOffset_Delta_Class_1001'].values.tolist()
size_train = df[:training_pt_1]['Size_class'].values.tolist()
size_test = df[training_pt_1+1:]['Size_class'].values.tolist()

In [24]:
data_path  = r'/soe/cchakrab/test_output/output_csv/SYSTOR/p3/'

path_train  = r"/soe/cchakrab/test_output/output_csv/SYSTOR/p3/delta_train.txt"

with open(path_train, 'w') as f:
    for item in data_train:
        f.write("%s " % item)
        
path_test  = r"/soe/cchakrab/test_output/output_csv/SYSTOR/p3/delta_test.txt"
with open(path_test, 'w') as f:
    for item in data_test:
        f.write("%s " % item)


path_train  = r"/soe/cchakrab/test_output/output_csv/SYSTOR/p3/size_train.txt"
with open(path_train, 'w') as f:
    for item in size_train:
        f.write("%s " % item)
        
path_test  = r"/soe/cchakrab/test_output/output_csv/SYSTOR/p3/size_test.txt"
with open(path_test, 'w') as f:
    for item in size_test:
        f.write("%s " % item)

In [ ]:
from keras.utils import to_categorical
from PIL import Image


min_size = df['Size'].min()


def get_data_generator(df, indices, for_training, batch_size=16):
    lba_delta, io_size = [], []
    while True:
        for i in indices:
            r = df.iloc[i]
            delta, size_io = r['ByteOffset_Delta_Class_1001'], r['Size']
            lba_delta.append(delta)
            io_size.append(size_io / min_size)
            
            if len(lba_delta) >= batch_size:
                yield np.array(lba_delta),  np.array(io_size)
                lba_delta, io_size = [], []
        if not for_training:
            break

In [ ]:
TRAIN_TEST_SPLIT = 0.7
count = 0
p = []
while(count <len(df)):
    p.append(count)
    count = count+1


train_up_to = int(len(df) * TRAIN_TEST_SPLIT)
train_idx = p[:train_up_to]
test_idx = p[train_up_to:]

# split train_idx further into training and validation set
train_up_to = int(train_up_to * 0.7)
train_idx, valid_idx = train_idx[:train_up_to], train_idx[train_up_to:]

In [ ]:
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

num_steps = 32
hidden_size = 1500 
use_dropout=True 
vocabulary_1 = 1001
vocabulary_2 = 259


# define two sets of inputs
inputA = Input(shape=(32,))
inputB = Input(shape=(32,))
# the first branch operates on the first input
x = Embedding(vocabulary_1, hidden_size, input_length=32)(inputA)
x = Model(inputs=inputA, outputs=x)
# the second branch opreates on the second input
y = Embedding(vocabulary_2, hidden_size, input_length=32)(inputB)
y = Model(inputs=inputB, outputs=y)
# combine the output of the two branches
combined = concatenate([x.output, y.output])
# # apply a FC layer and then a regression prediction on the
# # combined outputs
# z = Dense(2, activation="relu")(combined)
# z = Dense(1, activation="linear")(z)
# # our model will accept the inputs of the two branches and
# # then output a single value
# model = Model(inputs=[x.input, y.input], outputs=z)
# model.summary()



lstm1 = LSTM(hidden_size, return_sequences=True)(combined)
lstm2 = LSTM(hidden_size, return_sequences=True)(lstm1)

X= Dense(units=128, activation='relu')(lstm2)
iosize = Dense(units=1, activation='sigmoid', name='iosize')(X)

# create classification output
offset = Dense(units=1001, activation='relu', name='offset')(lstm2)


model = Model(inputs=[x.input, y.input], outputs=[iosize, offset])
model.compile(optimizer=Adam(), 
              loss={'iosize': 'mse', 'offset': 'categorical_crossentropy'},
              loss_weights={'iosize': 1., 'offset': 1},
              metrics={'iosize': 'mae', 'offset': 'accuracy'})
model.summary()



In [ ]:
from keras.callbacks import ModelCheckpoint

batch_size = 16
valid_batch_size = 16
train_gen = get_data_generator(df, train_idx, for_training=True, batch_size=batch_size)
valid_gen = get_data_generator(df, valid_idx, for_training=True, batch_size=valid_batch_size)

callbacks = [
    ModelCheckpoint("./model_checkpoint", monitor='val_loss')
]

history = model.fit_generator(train_gen,
                    steps_per_epoch=len(train_idx)//batch_size,
                    epochs=10,
                    verbose =1,
                    callbacks=callbacks,
                    validation_data=valid_gen,
                    validation_steps=len(valid_idx)//valid_batch_size)

In [ ]:
import time
start_time = time.time()

model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,
                        validation_data=test_data_generator.generate(),
                        validation_steps=len(train_data)//(batch_size*num_steps), callbacks=[checkpointer,monitor])

end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))